In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers
#!pip install sentence-piece

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Specify the device type (CUDA if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained model and tokenizer
model_name = 'microsoft/DialoGPT-medium'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.to(device)

# Load your .txt dataset
train_path = '/content/drive/MyDrive/Mangroves fine-tuning dataset.txt'
test_path = '/content/drive/MyDrive/Mangroves fine-tuning dataset.txt'

train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128
)

test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=test_path,
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# Set up the training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/results",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Create the Trainer and train
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:5

Step,Training Loss


TrainOutput(global_step=460, training_loss=1.6280825407608697, metrics={'train_runtime': 213.732, 'train_samples_per_second': 8.422, 'train_steps_per_second': 2.152, 'total_flos': 417915312537600.0, 'train_loss': 1.6280825407608697, 'epoch': 20.0})

In [ ]:
# Save fine-tuned model
model.save_pretrained('/content/drive/MyDrive/mangrove_finetuned_DialoGPT')

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model and tokenizer
tuned_model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/mangrove_finetuned_DialoGPT')
#tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#model = GPT2LMHeadModel.from_pretrained(model_name)

while True:
  # Tokenize input text
  input_text = input("Enter your text: ")
  c = input("You wanna continue the conversation, yet after? (y/n): ")
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  input_ids = input_ids.to(next(tuned_model.parameters()).device)

  # Generate response using the model
  output = tuned_model.generate(input_ids, max_length = 500)
  output_text = tokenizer.decode(output[0], skip_special_tokens=True)
  print(output_text)

  if c == "y":
    pass

  else:
    break

Enter your text: What is Salinity stress?
You wanna continue the conversation, yet after? (y/n): y


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is Salinity stress? Salt exclusion is a specialized feature of mangroves under salinity stress (Hogan et al., 2004).

Salinity stress is defined as the accumulation of salts in the soil that forms a barrier that prevents root growth and leads to premature senescence and ultimately leads to cell death (Ball and Farquhar, 2007). Salinity stress is defined as the accumulation of salts in the soil that forms a barrier that leads to cell death (Ball and Farquhar, 2007). The salinity stress of mangroves is mainly due to the deposition of excessive salts into the soil which eventually results in
Enter your text: How can mangroves help conserve freshwater in coastal areas??
You wanna continue the conversation, yet after? (y/n): n


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How can mangroves help conserve freshwater in coastal areas??
The leaves help in the sequestration of freshwater in the soil and the trunk helps in the conservation of the trunk biomass. Additionally, the trunk, along with the upper part of the leaves, help in the conservation of both freshwater and sedimentation.
2.2.4. Biomass yield in true mangroves

According to Naidoo and Von Willert (1995), greater assimilation and deposition of carbon from roots resulted in a faster and more efficient restoration of carbon under high salinity. Additionally, the trunk, upper part of the leaves, and and the hypocotyl capillary margin of the mangrove community support the restoration of carbon and other bioassiencies (Krauss and sequestration (Krauss and Von Willert).

Krauss and Von)).
Krauss and Von) have hypothesized that mangrove mangrove mangrove roots help in the sequestration and bioassimilation. They maintain the carbon and sequestration in the carbon and sequestration in the mangrove tissu

In [ ]:
import os
import zipfile

def zip_directory(output_filename, source_dir):
    with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file),
                           os.path.join(source_dir, '..')))

zip_directory('/content/mangrove_model1.zip', '/content/mangrove_model')


In [ ]:
from google.colab import files

# Replace 'your_zip_file.zip' with the actual name of your zipped file.
files.download('/content/mangrove_model1.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import tensorrt